In [39]:
import mysql.connector
import joblib
from datetime import datetime
import pandas as pd
db1 = mysql.connector.connect(host='localhost',user='root',password='martin',port=32000,database='proyect') 

In [40]:
mydb = db1.cursor()
query = ("select * from PROYECT")
mydb.execute(query)
myresult = mydb.fetchall()

In [41]:
df = pd.DataFrame(myresult)

In [42]:
df.columns = ['Text','Category']

In [43]:
df

,Text,Category
0,duracell aaa battery pack compatible select el...,"Connected Home & Housewares,Housewares,Househo..."
1,duracell aa v coppertop battery pack long last...,"Connected Home & Housewares,Housewares,Househo..."
2,duracell aa battery pack compatible select ele...,"Connected Home & Housewares,Housewares,Househo..."
3,energizer max battery aa pack pack aa alkaline...,"Connected Home & Housewares,Housewares,Househo..."
4,duracell c battery pack compatible select elec...,"Connected Home & Housewares,Housewares,Househo..."
...,...,...
50963,dell optiplex small form factor desktop intel ...,Computers & Tablets
50964,metra speaker connector select volkswagen vehi...,"Car Electronics & GPS,Car Installation Parts &..."
50965,dell optiplex small form factor desktop intel ...,Computers & Tablets
50966,dell optiplex small form factor desktop intel ...,Computers & Tablets


In [44]:
NameColumns = ['cat0' , 'cat1' , 'cat2', 'cat3', 'cat4', 'cat5']
cat = df['Category'].str.split(',', expand=True)
cat.columns = NameColumns

In [45]:
cat

,cat0,cat1,cat2,cat3,cat4,cat5
0,Connected Home & Housewares,Housewares,Household Batteries,None,None,None
1,Connected Home & Housewares,Housewares,Household Batteries,None,None,None
2,Connected Home & Housewares,Housewares,Household Batteries,None,None,None
3,Connected Home & Housewares,Housewares,Household Batteries,None,None,None
4,Connected Home & Housewares,Housewares,Household Batteries,None,None,None
...,...,...,...,...,...,...
50963,Computers & Tablets,None,None,None,None,None
50964,Car Electronics & GPS,Car Installation Parts & Accessories,None,None,None,None
50965,Computers & Tablets,None,None,None,None,None
50966,Computers & Tablets,None,None,None,None,None


In [8]:
cat = cat[cat.columns[0:3]]

In [9]:
cat = cat.replace('',None)

In [10]:
cate = cat.copy()

In [11]:
cat

,cat0,cat1,cat2
0,Connected Home & Housewares,Housewares,Household Batteries
1,Connected Home & Housewares,Housewares,Household Batteries
2,Connected Home & Housewares,Housewares,Household Batteries
3,Connected Home & Housewares,Housewares,Household Batteries
4,Connected Home & Housewares,Housewares,Household Batteries
...,...,...,...
50962,Computers & Tablets,None,None
50963,Computers & Tablets,None,None
50964,Car Electronics & GPS,Car Installation Parts & Accessories,None
50965,Computers & Tablets,None,None


In [ ]:
for i in cat:
    row = 0
    for category in cat[i]:
        if category!= 0:
         cat.loc[row,category] = 1
         row += 1

cat = cat.fillna(0)

In [13]:
df = pd.concat([df,cat],axis=1)

In [14]:
df = df.drop_duplicates()

In [15]:
X = df.Text
Y = df[df.columns[5:]]

In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


2022-07-31 17:57:27.406819: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-31 17:57:27.406879: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [17]:
X

0        duracell aaa battery pack compatible select el...
1        duracell aa v coppertop battery pack long last...
2        duracell aa battery pack compatible select ele...
3        energizer max battery aa pack pack aa alkaline...
4        duracell c battery pack compatible select elec...
                               ...                        
50959    amazon fire tv stick stream p content dual ban...
50960    proctor silex slice toaster oven white proctor...
50961    metra speaker connector select volkswagen vehi...
50962    dell optiplex small form factor desktop intel ...
50964    metra speaker connector select volkswagen vehi...
Name: Text, Length: 47644, dtype: object

In [18]:
#First Step -> Tokenize and padd
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>") #num_words : the maximum number of words to keep, based on word frequency (for the 45232 products information), #oov : out-of-vocabulary token, it is agregated to words which are not in the 10000 most frequent ones
tokenizer.fit_on_texts(X) # encoding words of information to integers, Updates internal vocabulary based on a list of sequences.
word_index = tokenizer.word_index
print("Amount of different words through the 45232 products information:",len(word_index))
sequences = tokenizer.texts_to_sequences(X)
print("Amount of sequences:", len(sequences)) #Should be equal to the amount of products 
print("Sequences Examples:",sequences[0], sequences[1], sequences[2], sequences[3], sequences[4]) #This is how the first 5 products info are represented, each number represent a word in "word_index" dict.
padded = pad_sequences(sequences, padding="post") #This takes the maxlen parameter as a referene and makes all the lists of the same lenght adding zeros on the left spaces, if nothins is pass ot will take the longest sequence as a reference
print("First Sequence padded: \n",padded[0])
print(len(padded[0]), len(padded[1]), len(padded[2])) #Should all be equal to 
print("Dataset Padded Shape:", padded.shape)

Amount of different words through the 45232 products information: 20548
Amount of sequences: 47644
Sequences Examples: [3436, 1636, 28, 59, 3, 13, 110, 49, 1636, 154, 5100, 10, 3017, 26, 59] [3436, 1336, 62, 1, 28, 59, 883, 1536, 431, 5100, 10, 3017, 26, 1710, 923, 155, 137, 148, 5101] [3436, 1336, 28, 59, 3, 13, 110, 49, 1336, 154, 5100, 10, 3017, 26, 59] [1388, 1139, 28, 1336, 59, 59, 1336, 2361, 28, 28, 6111, 23] [3436, 334, 28, 59, 3, 13, 110, 49, 334, 154, 5100, 10, 3017, 26, 59]
First Sequence padded: 
 [3436 1636   28   59    3   13  110   49 1636  154 5100   10 3017   26
   59    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]
59 59 59
Dataset Padded Shape: (47644, 59)


In [ ]:
joblib.dump(tokenizer,'tokenizer.plk')

In [19]:
# Splitting dataset in train/test set
X_train, X_test, y_train, y_test = train_test_split(padded,Y, 
                                                    test_size=0.3, 
                                                    random_state=0)

In [20]:
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalMaxPool1D, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import plot_model

In [21]:
num_classes = y_train.shape[1]
print(num_classes)
max_words = len(word_index) + 1
print(max_words)
maxlen = len(padded[0])
print(maxlen)

760
20549
59


In [22]:
from keras.layers import Activation, Flatten, Conv1D

In [23]:
optimizer = tf.keras.optimizers.Adam()

model = Sequential()
model.add(Embedding(max_words, 300, input_length=maxlen))
model.add(Dropout(0.21))
model.add(Conv1D(300, 3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPool1D()) # This layer creates a convolution kernel that is convolved with the layer input over a single spatial (or temporal) dimension to produce a tensor of outputs.
model.add(Dropout(0.21))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['Precision','Recall', tf.keras.metrics.AUC()])

callbacks = [
    ReduceLROnPlateau(),
    ModelCheckpoint(filepath='model-conv1d.h5', save_best_only=True)
]

model.summary()
model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 59, 300)           6164700   
                                                                 
 dropout (Dropout)           (None, 59, 300)           0         
                                                                 
 conv1d (Conv1D)             (None, 57, 300)           270300    
                                                                 
 global_max_pooling1d (Globa  (None, 300)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout_1 (Dropout)         (None, 300)               0         
                                                                 
 dense (Dense)               (None, 760)               228760    
                                                        

2022-07-31 17:57:32.317064: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-31 17:57:32.317087: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-31 17:57:32.317103: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (martin-MS-7C09): /proc/driver/nvidia/version does not exist
2022-07-31 17:57:32.317732: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [47]:
model.fit(X_train, y_train,
                    epochs=30,
                    batch_size=32,
                    validation_split=0.3,
                    callbacks=callbacks)

NameError: name 'model' is not defined

In [25]:
from sklearn.metrics import f1_score,recall_score,precision_score

In [26]:
predictions = model.predict(X_test, batch_size=32)

447/447 [==============================] - 2s 4ms/step


In [51]:
def get_standar_metrics(predictions, y_test):
  a = tranform_binary(predictions)
  print("Evaluation Metrics")
  print('-'*30)
  print("F1 Score:",f1_score(y_test, a, average="micro"))
  print("Recall Score:",recall_score(y_test, a, average="micro"))
  print("Precision Score:",precision_score(y_test, a, average="micro"))

  return f1_score(y_test, a, average="micro"),recall_score(y_test, a, average="micro"),precision_score(y_test, a, average="micro")

In [60]:
metrics = get_standar_metrics(predictions, y_test)

Evaluation Metrics
------------------------------
F1 Score: 0.9025376713554529
Recall Score: 0.8847719298245614
Precision Score: 0.9210314851340492


In [35]:

var = (str(round(metrics[0],2)),datetime.today())

In [20]:
import mysql.connector
db1 = mysql.connector.connect(host='localhost',user='root',password='martin',port=32000,database='proyect') 

In [37]:
mydb = db1.cursor()


query = """ INSERT INTO proyect.retrain 
            (f1,Date)
            VALUES (%s,%s) """
mydb.execute(query,var)
db1.commit()
